In [1]:
import os
import numpy as np
import natsort 
from scipy.interpolate import splrep, splev
import csv
import matplotlib.pyplot as plt

# Definite Constant
c_wid = 150e-6 # Channel Width # Unit : m
c_len = 10e-6 # Channel Length # Unit : m
k_ox = 3.9   # SiO2 산화막 유전상수
eps_vac = 8.854e-12 # Unit : F/m
t_ox = 1000e-10 # 산화막 두께 1000 aungstrom # Unit : m  
C_ox = (eps_vac * k_ox / t_ox) * 1e-4 # Unit : F/cm^2

# results = []
# pn=0

In [2]:
data_preprocess = np.load(r'D:\NMDL\졸업논문\KMeans\dataset_(20240520_종합).npz')['dataset']
data_Vg = data_preprocess[:, :, 2][0]

data = np.load(r'D:\NMDL\졸업논문\Kmeans\extract data after train\Extract dataset_(20240520_종합).npz')['dataset']
print(f'High Grade Samples : ', data.shape)

High Grade Samples :  (869, 301)


In [3]:
data_copy = data
Num = np.arange(1, len(data)+1).tolist()
X = data_Vg


Vth = []
Mobility = []
Ratio = []
SS = []
poor_index = []


for i in range(data.shape[0]) :
    # Threshold Voltage
    d_Vg = 0.2 # Vg = -40V to 20V , step_size = 0.2V

    sqrt_ID = []
    for j in range(len(data[i])) :
        if j == 0 :
            sqrt_ID.append(np.sqrt(data[i][j]))
    
        elif data[i][j] >= data[i][j-1] :
            sqrt_ID.append(np.sqrt(data[i][j]))
    
        else :
            sqrt_ID.append(np.sqrt(data[i][j-1]))
        
    d_sqrt_ID = []
    for j in range(len(data[i])) :
        if j == len(data[i]) - 1 :
            d_sqrt_ID.append(0 - sqrt_ID[j])
        else :
            d_sqrt_ID.append(sqrt_ID[j+1] - sqrt_ID[j])
        
    slope = []
    for j in range(len(data[i])) :
        slope.append(d_sqrt_ID[j] / d_Vg)
    

    X1 = X[np.argmax(slope)]
    Y1 = sqrt_ID[np.argmax(slope)]
    vth = X1 - (Y1 / np.max(slope))
    Vth.append(vth)
    
    # Saturation Mobility
    mobility = ((np.max(slope)**2) * 2 * c_len) / (c_wid * C_ox)
    Mobility.append(mobility)

    # On/Off Ratio
    ratio = np.max(data[i]) / np.min(data[i])
    Ratio.append(ratio)
    
    # Subthreshold Swing
    x_interpolation = np.linspace(min(X), max(X), len(X)*1000)
    y_spline = splrep(X, data[i], k = 1)      # k=1 선형보간
    y_interpolation = splev(x_interpolation, y_spline)
    
    lst = [1e-10, 1e-9, 1e-8, 1e-7]

    y_intercept_list = []
    for j in lst:
        if j < min(data[i]) or j > max(data[i]) :
            pass
        else :
            y_intercept_list.append(j)
            
    x_interpolation_list = []
    for y_intercept in y_intercept_list :
        x_intercept = np.where(y_interpolation <= y_intercept, y_intercept, y_interpolation)
        x_index = np.max(np.where(x_intercept == y_intercept))
        x_interpolation_list.append(x_interpolation[x_index])
        
    ss_list = []
    for l in range(len(x_interpolation_list)) :
        if l == (len(x_interpolation_list) - 1) :
            break
        else :
            ss_list.append(x_interpolation_list[l+1] - x_interpolation_list[l])
        
    ss = min(ss_list)
    SS.append(ss)
    
    
    num = Num[i]
    
    # print('%d \t %.2f \t %.4f \t %.2e \t %.3f' %(num, vth, mobility, ratio, ss))
    
    # Parameter 이상치 filter 후 출력
    if vth >= 9.00 or mobility >= 20:
        poor_index.append(i)
    else :
        print('%d \t %.2f \t %.4f \t %.2e \t %.3f' %(num, vth, mobility, ratio, ss))

1 	 6.36 	 9.0136 	 4.93e+08 	 0.397
2 	 8.28 	 12.6188 	 4.04e+08 	 0.330
8 	 6.92 	 12.9509 	 3.40e+08 	 0.321
9 	 6.95 	 14.8978 	 3.66e+08 	 0.278
14 	 7.39 	 14.6840 	 4.11e+08 	 0.427
30 	 4.75 	 10.9231 	 1.80e+09 	 0.283
31 	 4.92 	 10.8656 	 1.28e+09 	 0.257
32 	 4.43 	 10.3733 	 1.16e+09 	 0.291
33 	 4.72 	 10.2226 	 1.27e+09 	 0.253
34 	 4.44 	 9.4210 	 1.14e+09 	 0.193
35 	 5.20 	 12.1054 	 1.60e+09 	 0.305
36 	 4.22 	 11.4703 	 4.36e+08 	 0.179
37 	 4.98 	 11.4229 	 8.23e+08 	 0.314
38 	 4.86 	 11.2422 	 1.05e+09 	 0.269
39 	 4.69 	 10.2401 	 1.02e+09 	 0.240
40 	 5.32 	 11.6241 	 2.07e+09 	 0.314
41 	 5.42 	 12.0450 	 1.65e+09 	 0.324
42 	 5.32 	 11.6802 	 1.47e+09 	 0.396
43 	 5.40 	 11.4502 	 1.27e+09 	 0.384
44 	 5.15 	 11.4659 	 1.05e+09 	 0.397
45 	 4.59 	 10.3532 	 9.18e+08 	 0.357
46 	 5.03 	 12.0279 	 1.60e+09 	 0.229
47 	 5.42 	 12.2899 	 1.35e+09 	 0.284
48 	 5.84 	 11.9126 	 1.24e+09 	 0.307
49 	 5.90 	 11.9339 	 1.11e+09 	 0.301
50 	 5.48 	 11.3057 	 4.65e+09 

In [4]:
fil_data = np.delete(arr = data, obj = poor_index, axis = 0)

# DL 학습에 사용할 최종 필터링된 데이터
np.savez(r'D:\NMDL\졸업논문\DL\input dataset\Filtered_Dataset_(20240520 종합)', dataset = fil_data) 

In [5]:
# Chracteristics Parameter 이상치 filtering

import pandas as pd
# pd.set_option('display.max_rows', None) # dataframe row 생략 없이 출력

para_df = pd.DataFrame(data = zip(Vth, Mobility, SS, Ratio),
                       columns = ['Vth', 'Mobility', 'S-Slope', 'On/Off'],
                      )
# para_df.to_csv(r'D:\NMDL\졸업논문\Ground Truth\Ground Truth_(20240502 종합)_total.csv') # CSV Export

# filtering
para_df_fil = para_df.loc[(para_df['Vth'] < 9.0) & (para_df['Mobility'] < 20.0)] # filtered data
# para_df_outlier = para_df.loc[(para_df['Vth'] >= 10.0) | (para_df['Mobility'] >= 20.0)] # outlier data

# filter data Export
para_df_fil.to_csv(r'D:\NMDL\졸업논문\Ground Truth\Ground Truth_(20240520 종합).csv') # CSV Export
para_arr_fil = para_df_fil.to_numpy()
np.savez(r'D:\NMDL\졸업논문\Ground Truth\Ground Truth_(20240520 종합)', dataset = para_arr_fil) # NPZ Export


